# Experiment 059: Light Extrapolation Detection

**Goal:** Try lighter extrapolation detection parameters to reduce CV degradation.

**Changes from exp_058:**
- blend_strength: 0.3 → 0.1 (less aggressive blending)
- blend_threshold: 0.5 → 0.7 (more selective - only blend for very dissimilar solvents)
- Use Spange-based similarity instead of fragprint Tanimoto (aligns with model features)

**Hypothesis:** Lighter blending will preserve more of the base model's CV while still providing some intercept reduction.

In [1]:
# Imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC, abstractmethod
import tqdm
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

torch.set_default_dtype(torch.double)

# Data path for local execution
DATA_PATH = "/home/data"

print("Imports complete.")

Imports complete.


In [2]:
# Constants from official template
INPUT_LABELS_FULL_SOLVENT = [
    "Residence Time",
    "Temperature",
    "SOLVENT A NAME",
    "SOLVENT B NAME",
    "SolventB%",
]

INPUT_LABELS_SINGLE_SOLVENT = [
    "Residence Time",
    "Temperature",
    "SOLVENT NAME",
]

INPUT_LABELS_NUMERIC = [
    "Residence Time",
    "Temperature",
]

TARGET_LABELS = [
    "Product 2",
    "Product 3",
    "SM",
]

print("Constants defined.")

Constants defined.


In [3]:
# Data loading functions
def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    assert name in ["spange_descriptors", "acs_pca_descriptors", "drfps_catechol", "fragprints", "smiles"]
    features = pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)
    return features

# CV functions from official template
def generate_leave_one_out_splits(X, Y):
    """Generate all leave-one-out splits across the solvents."""
    all_solvents = X["SOLVENT NAME"].unique()
    for solvent_name in sorted(all_solvents):
        train_idcs_mask = X["SOLVENT NAME"] != solvent_name
        yield (
            (X[train_idcs_mask], Y[train_idcs_mask]),
            (X[~train_idcs_mask], Y[~train_idcs_mask]),
        )

def generate_leave_one_ramp_out_splits(X, Y):
    """Generate all leave-one-out splits across the solvent ramps."""
    all_solvent_ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    all_solvent_ramps = all_solvent_ramps.sort_values(by=["SOLVENT A NAME", "SOLVENT B NAME"])
    for _, solvent_pair in all_solvent_ramps.iterrows():
        train_idcs_mask = ~((X["SOLVENT A NAME"] == solvent_pair["SOLVENT A NAME"]) & 
                           (X["SOLVENT B NAME"] == solvent_pair["SOLVENT B NAME"]))
        yield (
            (X[train_idcs_mask], Y[train_idcs_mask]),
            (X[~train_idcs_mask], Y[~train_idcs_mask]),
        )

print("Data loading and CV functions defined.")

Data loading and CV functions defined.


In [4]:
# Base classes from official template
class SmilesFeaturizer(ABC):
    def __init__(self):
        raise NotImplementedError

    def featurize(self, X):
        raise NotImplementedError

class BaseModel(ABC):
    def __init__(self):
        pass

    def train_model(self, X_train, y_train):
        raise NotImplementedError

    def predict(self):
        raise NotImplementedError

print("Base classes defined.")

Base classes defined.


In [5]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)

print(f'Spange: {SPANGE_DF.shape}')
print(f'Solvents: {SPANGE_DF.index.tolist()}')

Spange: (26, 13)
Solvents: ['Cyclohexane', 'Ethyl Acetate', 'Acetic Acid', '2-Methyltetrahydrofuran [2-MeTHF]', '1,1,1,3,3,3-Hexafluoropropan-2-ol', 'IPA [Propan-2-ol]', 'Ethanol', 'Methanol', 'Ethylene Glycol [1,2-Ethanediol]', 'Acetonitrile', 'Water', 'Diethyl Ether [Ether]', 'MTBE [tert-Butylmethylether]', 'Dimethyl Carbonate', 'tert-Butanol [2-Methylpropan-2-ol]', 'DMA [N,N-Dimethylacetamide]', '2,2,2-Trifluoroethanol', 'Dihydrolevoglucosenone (Cyrene)', 'Decanol', 'Butanone [MEK]', 'Ethyl Lactate', 'Methyl Propionate', 'THF [Tetrahydrofuran]', 'Water.Acetonitrile', 'Acetonitrile.Acetic Acid', 'Water.2,2,2-Trifluoroethanol']


In [6]:
# Compute Spange-based similarity (cosine similarity on normalized descriptors)
def compute_spange_similarity_matrix(spange_df):
    """Compute pairwise cosine similarity matrix using Spange descriptors."""
    # Normalize each solvent's features
    from sklearn.preprocessing import normalize
    
    solvents = spange_df.index.tolist()
    features = spange_df.values
    
    # Normalize to unit vectors
    features_norm = normalize(features, axis=1)
    
    # Compute cosine similarity (dot product of normalized vectors)
    similarity_matrix = np.dot(features_norm, features_norm.T)
    
    return pd.DataFrame(similarity_matrix, index=solvents, columns=solvents)

# Compute similarity matrix using Spange descriptors
SIMILARITY_MATRIX = compute_spange_similarity_matrix(SPANGE_DF)
print(f"Similarity matrix shape: {SIMILARITY_MATRIX.shape}")
print(f"\nSample similarities for 'Acetonitrile':")
print(SIMILARITY_MATRIX.loc['Acetonitrile'].sort_values(ascending=False).head(5))

Similarity matrix shape: (26, 26)

Sample similarities for 'Acetonitrile':
Acetonitrile                        1.000000
DMA [N,N-Dimethylacetamide]         0.999372
Water.2,2,2-Trifluoroethanol        0.998276
Ethylene Glycol [1,2-Ethanediol]    0.994680
Methanol                            0.989066
Name: Acetonitrile, dtype: float64


In [7]:
# Light Extrapolation-aware model
class LightExtrapolationModel(BaseModel):
    """Model with LIGHT extrapolation detection.
    
    Key changes from exp_058:
    - blend_strength: 0.1 (was 0.3) - less aggressive
    - blend_threshold: 0.7 (was 0.5) - more selective
    - Uses Spange-based cosine similarity (was fragprint Tanimoto)
    """
    
    def __init__(self, data='single', blend_threshold=0.7, blend_strength=0.1):
        self.data = data
        self.blend_threshold = blend_threshold
        self.blend_strength = blend_strength
        self.spange_df = SPANGE_DF
        self.similarity_matrix = SIMILARITY_MATRIX
        self.models = None
        self.scaler = None
        self.train_mean = None
        self.train_solvents = None
    
    def _prepare_features(self, X):
        """Prepare features for the model."""
        # Numeric features with Arrhenius engineering
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.data == 'single':
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            return np.hstack([X_kinetic, X_spange])
        else:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            X_spange = A_spange * (1 - pct) + B_spange * pct
            return np.hstack([X_kinetic, pct, X_spange])
    
    def _get_max_similarity_to_training(self, test_solvent, train_solvents):
        """Get maximum similarity between test solvent and any training solvent."""
        if test_solvent not in self.similarity_matrix.index:
            return 0.0
        
        similarities = []
        for train_solvent in train_solvents:
            if train_solvent in self.similarity_matrix.columns:
                sim = self.similarity_matrix.loc[test_solvent, train_solvent]
                similarities.append(sim)
        
        if not similarities:
            return 0.0
        return max(similarities)
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        """Train CatBoost + XGBoost ensemble."""
        X_features = self._prepare_features(train_X)
        Y_np = train_Y.values
        
        self.train_mean = Y_np.mean(axis=0)
        
        if self.data == 'single':
            self.train_solvents = train_X["SOLVENT NAME"].unique().tolist()
        else:
            self.train_solvents = list(set(train_X["SOLVENT A NAME"].tolist() + train_X["SOLVENT B NAME"].tolist()))
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_features)
        
        # Train CatBoost models
        self.cat_models = []
        for i in range(3):
            model = CatBoostRegressor(
                iterations=500,
                depth=6,
                learning_rate=0.05,
                l2_leaf_reg=3.0,
                random_seed=42,
                verbose=False,
            )
            model.fit(X_scaled, Y_np[:, i])
            self.cat_models.append(model)
        
        # Train XGBoost models
        self.xgb_models = []
        for i in range(3):
            model = XGBRegressor(
                n_estimators=400,
                max_depth=5,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,
                reg_lambda=1.0,
                random_state=42,
                verbosity=0,
            )
            model.fit(X_scaled, Y_np[:, i])
            self.xgb_models.append(model)
    
    def predict(self, test_X):
        """Predict with light extrapolation-aware blending."""
        X_features = self._prepare_features(test_X)
        X_scaled = self.scaler.transform(X_features)
        
        # Get base predictions from ensemble
        cat_preds = np.column_stack([m.predict(X_scaled) for m in self.cat_models])
        xgb_preds = np.column_stack([m.predict(X_scaled) for m in self.xgb_models])
        
        # Ensemble: 60% CatBoost + 40% XGBoost
        base_preds = 0.6 * cat_preds + 0.4 * xgb_preds
        
        # Compute extrapolation score for each sample
        if self.data == 'single':
            test_solvents = test_X["SOLVENT NAME"].values
            extrapolation_scores = []
            for solvent in test_solvents:
                max_sim = self._get_max_similarity_to_training(solvent, self.train_solvents)
                extrapolation_scores.append(1.0 - max_sim)
            extrapolation_scores = np.array(extrapolation_scores).reshape(-1, 1)
        else:
            test_solvents_A = test_X["SOLVENT A NAME"].values
            test_solvents_B = test_X["SOLVENT B NAME"].values
            extrapolation_scores = []
            for solv_a, solv_b in zip(test_solvents_A, test_solvents_B):
                max_sim_a = self._get_max_similarity_to_training(solv_a, self.train_solvents)
                max_sim_b = self._get_max_similarity_to_training(solv_b, self.train_solvents)
                avg_sim = (max_sim_a + max_sim_b) / 2
                extrapolation_scores.append(1.0 - avg_sim)
            extrapolation_scores = np.array(extrapolation_scores).reshape(-1, 1)
        
        # Light blending - only for very dissimilar solvents
        # extrapolation_score > (1 - threshold) triggers blending
        # With threshold=0.7, only solvents with similarity < 0.3 trigger blending
        blend_weights = np.clip(
            (extrapolation_scores - (1 - self.blend_threshold)) / self.blend_threshold,
            0.0, 1.0
        ) * self.blend_strength
        
        # Final predictions
        final_preds = (1 - blend_weights) * base_preds + blend_weights * self.train_mean
        final_preds = np.clip(final_preds, 0.0, 1.0)
        
        return torch.tensor(final_preds, dtype=torch.double)

print("LightExtrapolationModel defined.")

LightExtrapolationModel defined.


In [8]:
# Quick test
print("Testing LightExtrapolationModel...")
X, Y = load_data("single_solvent")
print(f"Single solvent data: X={X.shape}, Y={Y.shape}")

# Test one fold
split_gen = generate_leave_one_out_splits(X, Y)
(train_X, train_Y), (test_X, test_Y) = next(split_gen)

test_solvent = test_X['SOLVENT NAME'].iloc[0]
print(f"Test solvent: {test_solvent}")

# Check similarity to training solvents
train_solvents = train_X['SOLVENT NAME'].unique().tolist()
max_sim = max([SIMILARITY_MATRIX.loc[test_solvent, s] for s in train_solvents if s in SIMILARITY_MATRIX.columns])
print(f"Max similarity to training: {max_sim:.4f}")
print(f"Extrapolation score: {1 - max_sim:.4f}")

model = LightExtrapolationModel(blend_threshold=0.7, blend_strength=0.1)
model.train_model(train_X, train_Y)
preds = model.predict(test_X)

print(f"Predictions shape: {preds.shape}")
print(f"Predictions sample: {preds[0]}")
print(f"Actual sample: {test_Y.iloc[0].values}")
print("Test passed!")

Testing LightExtrapolationModel...
Single solvent data: X=(656, 3), Y=(656, 3)
Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol
Max similarity to training: 0.9952
Extrapolation score: 0.0048


Predictions shape: torch.Size([37, 3])
Predictions sample: tensor([0.0196, 0.0142, 0.9353])
Actual sample: [0.01692854 0.02519112 0.95783321]
Test passed!


In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = LightExtrapolationModel(blend_threshold=0.7, blend_strength=0.1) # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

print(f"Single solvent predictions: {len(submission_single_solvent)}")
print(f"Unique folds: {submission_single_solvent['fold'].nunique()}")

0it [00:00, ?it/s]

1it [00:00,  1.05it/s]

2it [00:01,  1.10it/s]

3it [00:02,  1.09it/s]

4it [00:03,  1.09it/s]

...

22it [00:21,  1.08it/s]

23it [00:22,  1.07it/s]

24it [00:23,  1.06it/s]

24it [00:23,  1.04it/s]

Single solvent predictions: 656
Unique folds: 24


In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = LightExtrapolationModel(data='full', blend_threshold=0.7, blend_strength=0.1) # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

print(f"Full data predictions: {len(submission_full_data)}")
print(f"Unique folds: {submission_full_data['fold'].nunique()}")

0it [00:00, ?it/s]

1it [00:01,  1.43s/it]

2it [00:02,  1.51s/it]

3it [00:04,  1.47s/it]

4it [00:05,  1.42s/it]

...

11it [00:15,  1.36s/it]

12it [00:16,  1.35s/it]

13it [00:18,  1.36s/it]

13it [00:18,  1.39s/it]

Full data predictions: 1227
Unique folds: 13


In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

print(f"Submission saved to /home/submission/submission.csv")
print(f"Total rows: {len(submission)}")

Submission saved to /home/submission/submission.csv
Total rows: 1883


In [ ]:
# Calculate CV for logging
print("\n" + "="*60)
print("CV CALCULATION")
print("="*60)

# Single solvent CV
X, Y = load_data("single_solvent")
fold_mses = []

for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X, Y)):
    model = LightExtrapolationModel(blend_threshold=0.7, blend_strength=0.1)
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mse = np.mean((preds - test_Y.values) ** 2)
    fold_mses.append(mse)
    if fold_idx % 5 == 0:
        print(f"  Fold {fold_idx}: MSE = {mse:.6f}")

single_cv = np.mean(fold_mses)
single_std = np.std(fold_mses)
print(f"\nSingle solvent CV MSE: {single_cv:.6f} ± {single_std:.6f}")

# Full data CV
X, Y = load_data("full")
full_fold_mses = []

for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_ramp_out_splits(X, Y)):
    model = LightExtrapolationModel(data='full', blend_threshold=0.7, blend_strength=0.1)
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mse = np.mean((preds - test_Y.values) ** 2)
    full_fold_mses.append(mse)
    print(f"  Fold {fold_idx}: MSE = {mse:.6f}")

full_cv = np.mean(full_fold_mses)
full_std = np.std(full_fold_mses)
print(f"\nFull data CV MSE: {full_cv:.6f} ± {full_std:.6f}")

print(f"\nFINAL CV FOR LOGGING: {single_cv:.6f}")

In [ ]:
# Summary
print("\n" + "="*60)
print("EXPERIMENT 059: LIGHT EXTRAPOLATION DETECTION SUMMARY")
print("="*60)

print("\nCHANGES FROM EXP_058:")
print("  - blend_strength: 0.3 → 0.1 (less aggressive)")
print("  - blend_threshold: 0.5 → 0.7 (more selective)")
print("  - Similarity: Fragprint Tanimoto → Spange cosine (aligns with model features)")

print(f"\nCV SCORES:")
print(f"  Single solvent: {single_cv:.6f} ± {single_std:.6f}")
print(f"  Full data: {full_cv:.6f} ± {full_std:.6f}")

# Compare to exp_058
print(f"\nCOMPARISON TO EXP_058:")
print(f"  exp_058 CV: 0.011541")
print(f"  exp_059 CV: {single_cv:.6f}")
print(f"  Improvement: {(0.011541 - single_cv) / 0.011541 * 100:.1f}%")

# Compare to best CV
print(f"\nCOMPARISON TO BEST CV:")
print(f"  Best CV: 0.008092")
print(f"  exp_059 CV: {single_cv:.6f}")
print(f"  Gap: {(single_cv - 0.008092) / 0.008092 * 100:.1f}%")

# Predicted LB
predicted_lb = 4.31 * single_cv + 0.0525
print(f"\nPREDICTED LB (using CV-LB relationship): {predicted_lb:.4f}")
print(f"  Best LB so far: 0.0877")
print(f"  Target: 0.0347")